# Episode 20

In a desperate attempt to regain some composure following a cheese migration incident, we note the deprecation of `pgen` in the CPython implementation, and then continue to assume nothing is wrong.  After all, using LL(1) pushdown automata is a perfectly fine method of parsing source code, and will continue to live in the heart and minds of countless pre-3.9 Pythonistas...

In [1]:
import parser

/var/folders/q7/ylhf40fn5kq9xrbfmsl61q4h0000gn/T/ipykernel_31067/1112699797.py:1: DeprecationWarning: The parser module is deprecated and will be removed in future versions of Python
  import parser


In [2]:
%load_ext metamagics

In [3]:
def ignore_this_after_3_9(cell):
    return parser.st2tuple(parser.suite(cell))

In [4]:
%%mycell ignore_this_after_3_9
1+2-3
A=42

(257,
 (269,
  (270,
   (271,
    (272,
     (274,
      (306,
       (310,
        (311,
         (312,
          (313,
           (316,
            (317,
             (318,
              (319,
               (320,
                (321, (322, (323, (324, (325, (2, '1')))))),
                (14, '+'),
                (321, (322, (323, (324, (325, (2, '2')))))),
                (15, '-'),
                (321, (322, (323, (324, (325, (2, '3'))))))))))))))))))),
   (4, ''))),
 (269,
  (270,
   (271,
    (272,
     (274,
      (306,
       (310,
        (311,
         (312,
          (313,
           (316,
            (317,
             (318,
              (319,
               (320, (321, (322, (323, (324, (325, (1, 'A'))))))))))))))))),
     (22, '='),
     (274,
      (306,
       (310,
        (311,
         (312,
          (313,
           (316,
            (317,
             (318,
              (319,
               (320, (321, (322, (323, (324, (325, (2, '42'))))))))))))))))))),
   

In [5]:
import io
from lib2to3.pgen2 import pgen, driver
from typing import Callable, Dict

class Bindify:
    class Binder:
        def __init__(self, parent: 'Bindify', binding_name: str):
            self.parent = parent
            self.binding_name = binding_name

        def __call__(self, cell, shell):
            result = self.parent.magic_fun(cell, shell)
            shell.user_global_ns[self.binding_name] = result
            return result

    def __init__(self, magic_fun: Callable):
        self.binders: Dict[str, Binder] = {}
        self.magic_fun = magic_fun

    def __getitem__(self, name: str):
        if name in self.binders:
            return self.binders[name]
        binder = Bindify.Binder(self, name)
        self.binders[name] = binder
        return binder
        
    def __call__(self, cell, shell):
        return self.magic_fun(cell, shell)

def _pgenify(cell, _):
    generator = pgen.ParserGenerator('<cell-magic-string>', io.StringIO(cell))
    return generator.make_grammar()

pgenify = Bindify(_pgenify)

In [7]:
%%mycell pgenify['calc_grammar']

statements: (statement NEWLINE)* ENDMARKER
statement: assign | expr
assign: NAME '=' expr
expr: atom [op expr]
atom: NAME | NUMBER
op: '+' | '-'

ValueError: rule statement is ambiguous; NAME is in the first sets of expr as well as assign

In [8]:
%%mycell pgenify['calc_grammar']

statements: (statement NEWLINE)* ENDMARKER
statement: expr ['=' expr]
expr: atom [op expr]
atom: NAME | NUMBER
op: '+' | '-'

In [9]:
import nbimport
import Episode16
Tree = Episode16.Tree
from lib2to3 import pytree

calc_driver = driver.Driver(calc_grammar, pytree.convert)
calc_driver.parse_string('1 + 2 - 3\n')

Node(file_input, [Node(and_test, [Leaf(2, '1'), Leaf(14, '+'), Node(and_test, [Leaf(2, '2'), Leaf(15, '-'), Leaf(2, '3')])]), Leaf(4, '\n'), Leaf(0, '')])

In [10]:
def pytree_to_tree(node):
    if isinstance(node, pytree.Leaf):
        return Tree(node, [])
    else:
        return Tree(node.type, [pytree_to_tree(child) for child in node.children])

def _calcify(cell, _):
    return pytree_to_tree(calc_driver.parse_string(cell))

calcify = Bindify(_calcify)

In [11]:
%%mycell calcify['tree']
1 + 2 - 3
A = 42

Tree(contents=256, children=[Tree(contents=258, children=[Tree(contents=Leaf(2, '1'), children=[]), Tree(contents=Leaf(14, '+'), children=[]), Tree(contents=258, children=[Tree(contents=Leaf(2, '2'), children=[]), Tree(contents=Leaf(15, '-'), children=[]), Tree(contents=Leaf(2, '3'), children=[])])]), Tree(contents=Leaf(4, '\n'), children=[]), Tree(contents=260, children=[Tree(contents=Leaf(1, 'A'), children=[]), Tree(contents=Leaf(22, '='), children=[]), Tree(contents=Leaf(2, '42'), children=[])]), Tree(contents=Leaf(4, '\n'), children=[]), Tree(contents=Leaf(0, ''), children=[])])

In [12]:
Episode16.tree_to_tuple(tree)

(256,
 [(258,
   [(Leaf(2, '1'), []),
    (Leaf(14, '+'), []),
    (258, [(Leaf(2, '2'), []), (Leaf(15, '-'), []), (Leaf(2, '3'), [])])]),
  (Leaf(4, '\n'), []),
  (260, [(Leaf(1, 'A'), []), (Leaf(22, '='), []), (Leaf(2, '42'), [])]),
  (Leaf(4, '\n'), []),
  (Leaf(0, ''), [])])

In [13]:
calc_grammar.report()

s2n
{'atom': 257, 'expr': 258, 'op': 259, 'statement': 260, 'statements': 256}
n2s
{256: 'statements', 257: 'atom', 258: 'expr', 259: 'op', 260: 'statement'}
states
[[[(1, 1), (2, 2)], [(0, 1)], [(3, 0)]],
 [[(4, 1), (5, 1)], [(0, 1)]],
 [[(6, 1)], [(7, 2), (0, 1)], [(8, 3)], [(0, 3)]],
 [[(9, 1), (10, 1)], [(0, 1)]],
 [[(8, 1)], [(11, 2), (0, 1)], [(8, 3)], [(0, 3)]]]
dfas
{256: ([[(1, 1), (2, 2)], [(0, 1)], [(3, 0)]], {1: 1, 4: 1, 5: 1}),
 257: ([[(4, 1), (5, 1)], [(0, 1)]], {4: 1, 5: 1}),
 258: ([[(6, 1)], [(7, 2), (0, 1)], [(8, 3)], [(0, 3)]], {4: 1, 5: 1}),
 259: ([[(9, 1), (10, 1)], [(0, 1)]], {9: 1, 10: 1}),
 260: ([[(8, 1)], [(11, 2), (0, 1)], [(8, 3)], [(0, 3)]], {4: 1, 5: 1})}
labels
[(0, 'EMPTY'),
 (0, None),
 (260, None),
 (4, None),
 (1, None),
 (2, None),
 (257, None),
 (259, None),
 (258, None),
 (14, None),
 (15, None),
 (22, None)]
start 256


## A Word of Warning

There is a bug in the generated parser where the `convert` argument is optional, but when left unused, the raw representation will be a tuple and not an object.

Therefore, this doesn't work:

In [14]:
bad_driver = driver.Driver(calc_grammar)
bad_driver.parse_string('1 + 2 - 3\n')

AttributeError: 'tuple' object has no attribute 'used_names'

In [17]:
class Tree2:
    def __init__(self, contents, children):
        self.contents = contents
        self.children = children

    def to_tuple(self):
        return self.contents, [child.to_tuple() for child in self.children]

def ohop_convert(grammar, raw_node):
    type, value, context, children = raw_node
    if children or type in grammar.number2symbol:
        return Tree2(type, children)
    else:
        return Tree2((type, value), [])

better_driver = driver.Driver(calc_grammar, ohop_convert)
better_driver.parse_string('1 + 2 - 3\n').to_tuple()

(256,
 [(260,
   [(258,
     [(257, [((2, '1'), [])]),
      (259, [((14, '+'), [])]),
      (258,
       [(257, [((2, '2'), [])]),
        (259, [((15, '-'), [])]),
        (258, [(257, [((2, '3'), [])])])])])]),
  ((4, '\n'), []),
  ((0, ''), [])])

In [18]:
def ohop_convert2(grammar, raw_node):
    type, value, context, children = raw_node
    if children or type in grammar.number2symbol:
        return Tree2(type if type not in grammar.number2symbol else grammar.number2symbol[type], children)
    else:
        return Tree2((type, value), [])

In [19]:
yet_another_driver = driver.Driver(calc_grammar, ohop_convert2)
yet_another_driver.parse_string('1 + 2 - 3\nA = 42\n').to_tuple()

('statements',
 [('statement',
   [('expr',
     [('atom', [((2, '1'), [])]),
      ('op', [((14, '+'), [])]),
      ('expr',
       [('atom', [((2, '2'), [])]),
        ('op', [((15, '-'), [])]),
        ('expr', [('atom', [((2, '3'), [])])])])])]),
  ((4, '\n'), []),
  ('statement',
   [('expr', [('atom', [((1, 'A'), [])])]),
    ((22, '='), []),
    ('expr', [('atom', [((2, '42'), [])])])]),
  ((4, '\n'), []),
  ((0, ''), [])])